# Top commodities in trade between partners

This notebook will collect data on the top commodities in trade between partners for a given period.

Partners can be a single country or a group of countries



## Setup 

See [here](0-comtrade-setup-first.ipynb) for instructions on how to obtain an API Key.
Without APIKEY results are limited to a maximum of 500 rows.


In [1]:
import comtradetools as comtradetools

comtradetools.setup()
APIKEY = comtradetools.get_api_key()
comtradetools.init(APIKEY, force_init=False)

INFO:root:Loading HS codes from support/harmonized-system.csv


## Top commodities in trade between partners

Obtains main commodities in trade between partners for a given period, using 2 digit HS codes.

For the meaning of codes see: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx


### TODO

- [ ] Permitir escolher os reporter e partners
- [ ] compatibilizar com o formato de report da AICEP https://myaicep.portugalexporta.com/mercados-internacionais/cn/china?setorProduto=-1
- [ ] há problemas de duplicação de linhas nas listagem de detalhes de commodities, provavlmente por custom code, ou mot code.
  

Define:
* `reporter` - country or countries providing the data
* `partner` - country or countries trading with the reporter
* `years_of_interest` - year or period for which data is requested
* `rank_filter` - number of top commodities to be analyzed
* `flowCode` M for imports, X for exports, M,X for both

In [2]:
import comtradetools as comtradetools

reporterDesc = 'China'
reporterCode = comtradetools.encode_country(reporterDesc)
partnerDesc = 'PLP'
partnerCode = comtradetools.m49_plp_list
year_start = 2003
year_end = 2023
years_of_interest = comtradetools.year_range(year_start,year_end+1)
rank_filter = 5  # número de importações mais relevantes
flowCode = 'M,X'


In [3]:
import comtradetools as comtrade
import pandas as pd



pco_cols = ['reporterDesc','partnerDesc','refYear','pyf_rank','cmdCode','cmdDesc',
            'flowCode','primaryValue','pyf_sum','pyf_perc']

periods = comtrade.split_period(years_of_interest,12)
df = pd.DataFrame()
for period in periods:
    print(f"Fetching data for period {period}")

    temp = comtrade.getFinalData(APIKEY,
                                typeCode="C",# C for commodities, S for Services
                                freqCode="A",# (freqCode) A for annual and M for monthly
                                flowCode=flowCode,
                                cmdCode="AG2",
                                reporterCode=reporterCode,
                                partnerCode=partnerCode,
                                period=period,
                                partner2Code=0,
                                clCode="HS",
                                customsCode='C00',
                                motCode=0,
                                includeDesc=True,
                                cache=True,
                                use_alternative=False
                                )
    df = pd.concat([df,temp], ignore_index=True)

# Subtotal by partner, year (not flow?)
pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
# Determine rank of each partner in import and export
pco['pyf_rank'] = pco.groupby(['partnerDesc','refYear','flowCode'])["primaryValue"].rank(method="dense", ascending=False)
pco['pyf_sum'] = pco.groupby(['partnerDesc','refYear','flowCode'])["primaryValue"].transform('sum')
pco['pyf_perc'] = pco['primaryValue'] / pco['pyf_sum']
# convert rank column to int
pco['pyf_rank'] = pco['pyf_rank'].astype(int)

# filter top partners
pco_tops = pco[pco['pyf_rank'] <= rank_filter]
cmdCodes_tops = pco_tops['cmdCode'].unique()
print(f"Product categories involved in exchanges with top partners:\n {cmdCodes_tops}")




INFO:root:Calling getFinalData for period 2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014


Fetching data for period 2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014


INFO:root:Calling getFinalData for period 2015,2016,2017,2018,2019,2020,2021,2022,2023,2024


Fetching data for period 2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Product categories involved in exchanges with top partners:
 ['27' '85' '64' '87' '52' '84' '25' '71' '39' '20' '90' '40' '73' '44'
 '03' '72' '74' '76' '94' '68' '62' '22' '01' '26' '63' '12' '47' '15'
 '29' '54' '41' '17' '02' '89' '38' '10' '95' '70' '34' '60' '61' '09'
 '16' '69' '99' '92' '49' '82' '48' '65' '97' '33' '30' '56' '08' '04'
 '21' '55' '96' '28' '31' '86' '45' '42' '83' '06' '07' '32' '14' '91'
 '78' '46']


In [4]:
# show
pd.options.display.max_colwidth=100
pd.options.display.max_rows=1600
pd.options.display.float_format = '{:,.2f}'.format

pco_tops_sorted = pco_tops[pco_cols].set_index(['reporterDesc','partnerDesc','flowCode','refYear','pyf_rank']).sort_index()
pco_tops_sorted.head(50).style.format(comtrade.make_format(list(pco_tops_sorted.columns)))


## Output to Excel file

In [5]:
# Prepare file name for Excel output

real_period = pco_tops_sorted.index.get_level_values('refYear').unique()
start_year=min(real_period)
end_year=max(real_period)
period=f"{start_year}-{end_year}"

filename_note=f"{period}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/{reporterDesc}_{partnerDesc}_tops_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)
# Save data
sheet_name = "comtrade"
pco_tops_sorted.reset_index().to_excel(excel_file, sheet_name=sheet_name, index=True, startrow=2)
# Adjust column width
comtrade.excel_col_autowidth(pco_tops_sorted, excel_file,sheet_name)

# format column primaryValue as currency
comtrade.excel_format_currency(pco_tops_sorted, excel_file,sheet_name, columns=['primaryValue'],width=20)

# Write title in first row TBD
excel_file_title = f"Commodities {reporterDesc}-{partnerDesc}, top {rank_filter} (M=Import, X=Export), {period} USD"
print(excel_file_title)
excel_file.close()
print("Saved to:",excel_file_name)


Commodities China-PLP, top 5 (M=Import, X=Export), 2003-2022 USD
Saved to: ./reports/China_PLP_tops_2003-2022_M_X.xlsx


## Analysis of variations in top commodities flows over time

The next cell provides a function that will analyze the variations in the top commodities flows over time.

For each combination of Reporter, Partner Year and Flow, the function will produce a list of the top commodities in trade between partners for a given period, using 2 digit HS codes.
```
China Brazil M
2003   12,26,72 | 12=1,683,531,130.00 (100%) | 26=1,369,127,929.00 (100%) | 72=745,347,034.00 (100%) | 
        -> 26 Ores, slag and ash
        -> 12 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
        -> 72 Iron and steel
2015 ! 12,26,27 | 12=16,887,916,146.00  (1003%) | 26=12,653,641,892.00  (924%) | 27=5,297,996,353.00  (new:27) | 
        -> 27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
2016 = 12,26,27 | 12=15,551,903,256.00 (92%) | 26=13,339,552,748.00 (105%) | 27=6,041,262,180.00 (114%) | 
2017 = 12,26,27 | 12=20,916,408,256.00 (134%) | 26=18,261,973,405.00 (137%) | 27=9,189,149,994.00 (152%) | 
2018 = 12,26,27 | 12=28,843,062,797.00 (138%) | 26=19,307,536,897.00 (106%) | 27=16,484,023,407.00 (179%) | 
12 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
26 Ores, slag and ash
27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
72 Iron and steel
```
Each year is followed by a character that indicates the variation in the top commodities from the previous year:
* ! : change in one of the top commodities
* = : same top commodities as the previous year, same relative importance
* ≈ : same top commodities as the previous year, different relative importance

Change `consider_first` to set the number of top commodities to be examined.

In [ ]:
consider_first = 3
for reporter in pco_tops_sorted.index.get_level_values(0).unique():
    for partner in pco_tops_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 

        for flow in pco_tops_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            print("Percentages show change of commodity value compared to previous year")
            previous_pattern = []
            all_commodities = set() # all commodities for this partner for this flow
            all_commodities_desc = set() # all commodities for this partner for this flow descriptions
            previous_years_values = dict()
            for year in pco_tops_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique():
                print(year, end=' ')
                top_codes = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                top_codes_desc = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                top_primary_value = pco_tops_sorted.loc[(reporter,partner,flow,year)]['primaryValue'].values.flatten().tolist()[0:consider_first]
                top_table = {code: {"desc":desc, "value":value, "year": year} for code,desc,value in zip(top_codes,top_codes_desc,top_primary_value)}
                codes_desc = {f"{code} {desc}" for code,desc in zip(top_codes,top_codes_desc)}
                # make set of tuples top5_codes and top5_codes_desc
                new_commodities = set(top_codes).difference(all_commodities)
                all_commodities = all_commodities.union(set(top_codes))
                all_commodities_desc = all_commodities_desc.union(codes_desc)
                pattern = ','.join(top_codes)
                if len(previous_pattern) == 0:
                    previous_pattern = top_codes
                    # update previous_years_values with top5_table
                    previous_years_values.update(top_table)
                    print(' ', pattern, end=' | ')
                    for cmd in top_codes:
                        print(f"{cmd}={top_table[cmd]['value']:,.2f} ({top_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                    for cmd in new_commodities:
                        print( " "*13+"  ->", cmd,top_table[cmd]['desc'])
                elif set(top_codes) == set(previous_pattern):
                    if top_codes == previous_pattern:
                        print('=', pattern, end=' | ')
                    else:
                        print('≈', pattern, end=' | ')
                    for cmd in top_codes:
                        print(f"{cmd}={top_table[cmd]['value']:,.2f} ({top_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                else:
                    print('!', pattern, end=' | ')
                    for cmd in top_codes:
                        print(f"{cmd}={top_table[cmd]['value']:,.2f} ", end=' ')
                        if cmd in previous_years_values.keys():
                            print(f"({top_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                        else:
                            print(f"(new:{cmd})", end=' | ')
                    print()
                    if len(new_commodities) > 0:
                        for cmd in new_commodities:
                            print( " "*13+"  ->", cmd, top_table[cmd]['desc'])
                       
                previous_pattern = top_codes
                previous_years_values.update(top_table)
            for cmd in sorted(all_commodities_desc):
                print( "    ",cmd)
            print(80*"-")



# AQUI

## Detalhe do tipo de produtos comercializados entre os PLP e a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada um das categorias

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [10]:
years = f"{year_start},{year_end}"
years

'2003,2023'

In [20]:
import comtradetools
import time
import pandas as pd


consider_first = 3
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCodeAG2','motDesc','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

detail_commodity_codes = pd.DataFrame()
for reporter in pco_tops_sorted.index.get_level_values(0).unique():
    for partner in pco_tops_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 

        for flow in pco_tops_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            years = sorted(pco_tops_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique())
            for year in [years[0],years[-1]]:
                print(year, end=' ')
                top_codes = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                top_codes_desc = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                time.sleep(5)
                df = comtradetools.getFinalData(
                    APIKEY,
                    typeCode="C",# C for commodities, S for Services
                    freqCode="A",# (freqCode) A for annual and M for monthly
                    flowCode=flow,
                    clCode="HS",
                    #cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                    cmdCode=None, # This gives all the commodities for the country and year 
                    reporterCode=comtradetools.encode_country(reporter),
                    partnerCode=comtradetools.encode_country(partner),
                    period=str(year),
                    partner2Code=0,
                    motCode=0,
                    customsCode='C00',
                    includeDesc=True,
                    cache=True,
                    use_alternative=False
                    )
                # create a column with the first 2 digits of the commodity code
                df['cmdCodeAG2'] = df.cmdCode.str[0:2]
                # filter the detail commodity codes by the top 5
                df = df[df['cmdCodeAG2'].isin(top_codes)]
                print(df['cmdCodeAG2'].unique(),df['cmdCode'].unique())
                detail_commodity_codes = pd.concat([detail_commodity_codes,df], ignore_index=True)

China Angola M
2003 

INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003


['25' '27' '71'] ['25' '2516' '251611' '27' '2709' '270900' '71' '7102' '710231']
2022 

INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022


['25' '27' '74'] ['25' '2506' '250610' '250620' '2515' '251511' '251512' '2516' '251611'
 '251612' '2530' '253010' '253090' '27' '2709' '270900' '2711' '271112'
 '271113' '2715' '271500' '74' '7403' '740312' '740313' '740319' '740321'
 '7404' '740400' '7408' '740819']
China Angola X
2003 

...

['09' '27' '44'] ['09' '0901' '090111' '27' '2709' '270900' '44' '4403' '440342' '4407'
 '440723' '4421' '442199']
China Timor-Leste X
2003 

INFO:root:Calling getFinalData for period 2003


['85' '87' '99'] ['85' '8504' '850410' '8507' '850720' '8513' '851310' '8514' '851440'
 '8516' '851660' '851671' '8517' '851730' '8525' '852520' '8532' '853222'
 '8536' '853661' '853669' '8538' '853890' '8539' '853929' '853931'
 '853939' '853990' '8543' '854390' '8544' '854411' '87' '8711' '871120'
 '8712' '871200' '8714' '871419' '871496' '99' '9999' '999999']
2022 

INFO:root:Calling getFinalData for period 2022


['72' '73' '84'] ['72' '7208' '720840' '720851' '720852' '720853' '720854' '720890' '7209'
 '720926' '720927' '7210' '721030' '721041' '721049' '721061' '721070'
 '7211' '721119' '721190' '7212' '721220' '721230' '7213' '721310'
 '721391' '7214' '721410' '721420' '721491' '721499' '7215' '721590'
 '7216' '721610' '721621' '721631' '721632' '721633' '721650' '721661'
 '721669' '721691' '721699' '7217' '721710' '721720' '721790' '7219'
 '721934' '721935' '7220' '722090' '7222' '722240' '7226' '722699' '7227'
 '722790' '7228' '722820' '722830' '722840' '722850' '7229' '722990' '73'
 '7302' '730240' '7304' '730419' '730423' '730439' '730441' '730449'
 '7305' '730590' '7306' '730630' '730640' '730650' '730661' '730669'
 '730690' '7307' '730719' '730721' '730722' '730729' '730791' '730792'
 '730799' '7308' '730810' '730820' '730830' '730840' '730890' '7309'
 '730900' '7310' '731010' '731021' '731029' '7311' '731100' '7312'
 '731210' '731290' '7313' '731300' '7314' '731419' '731420' '731431'


In [29]:
# show more rows
pd.set_option('display.max_rows', 1000)
# select country
partner_of_interest = 'Angola'
detail_commodity_codes[detail_commodity_codes['partnerDesc']==partner_of_interest]\
    [['reporterDesc','partnerDesc','flowCode','refYear','cmdCode','partner2Code','partner2Desc','motCode','customsCode','isAggregate','primaryValue','cmdDesc']]\
        .sort_values(by=['reporterDesc','partnerDesc','refYear','flowCode','cmdCode']).head(100)

,reporterDesc,partnerDesc,flowCode,refYear,cmdCode,partner2Code,partner2Desc,motCode,customsCode,isAggregate,primaryValue,cmdDesc
0,China,Angola,M,2003,25,0,World,0,C00,False,"232,059.00","Salt; sulphur; earths, stone; plastering materials, lime and cement"
1,China,Angola,M,2003,2516,0,World,0,C00,False,"232,059.00","Granite, porphyry, basalt, sandstone, other monumental and building stone, whether or not roughl..."
2,China,Angola,M,2003,251611,0,World,0,C00,False,"232,059.00",Granite; crude or roughly trimmed
3,China,Angola,M,2003,27,0,World,0,C00,False,"2,205,654,993.00","Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w..."
4,China,Angola,M,2003,2709,0,World,0,C00,False,"2,205,654,993.00",Petroleum oils and oils obtained from bituminous minerals; crude
5,China,Angola,M,2003,270900,0,World,0,C00,False,"2,205,654,993.00","Oils; petroleum oils and oils obtained from bituminous minerals, crude"
6,China,Angola,M,2003,71,0,World,0,C00,False,"47,397.00","Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with prec..."
7,China,Angola,M,2003,7102,0,World,0,C00,False,"47,397.00","Diamonds, whether or not worked, but not mounted or set"
8,China,Angola,M,2003,710231,0,World,0,C00,False,"47,397.00","Diamonds; non-industrial, unworked or simply sawn, cleaved or bruted, but not mounted or set"
40,China,Angola,X,2003,64,0,World,0,C00,False,"11,918,892.00",Footwear; gaiters and the like; parts of such articles


In [23]:
# show more rows
pd.set_option('display.max_rows', 1000)
detail_commodity_codes[['refYear','cmdCode','partner2Code','partner2Desc','motCode','customsCode','isAggregate','primaryValue','cmdDesc']].sort_values(by=['refYear','cmdCode','partner2Code']).head(100)

,refYear,cmdCode,partner2Code,partner2Desc,motCode,customsCode,isAggregate,primaryValue,cmdDesc
4200,2003,03,0,World,0,C00,False,"5,457.00","Fish and crustaceans, molluscs and other aquatic invertebrates"
4201,2003,0306,0,World,0,C00,False,"5,457.00","Crustaceans, in shell or not, live, fresh, chilled, frozen, dried, salted or in brine; crustacea..."
4202,2003,030613,0,World,0,C00,False,"5,457.00","Crustaceans; shrimps and prawns, frozen (whether in shell or not, whether or not cooked by steam..."
4203,2003,08,0,World,0,C00,False,"25,563.00","Fruit and nuts, edible; peel of citrus fruit or melons"
4204,2003,0806,0,World,0,C00,False,"25,563.00",Grapes; fresh or dried
4205,2003,080610,0,World,0,C00,False,"25,563.00","Fruit, edible; grapes, fresh"
3083,2003,10,0,World,0,C00,False,"573,895.00",Cereals
3974,2003,10,0,World,0,C00,False,"3,587,400.00",Cereals
4239,2003,10,0,World,0,C00,False,"4,406,997.00",Cereals
4240,2003,1001,0,World,0,C00,False,"1,199,920.00",Wheat and meslin


Temos que identificar as linhas com valor agregado porque
os dados não tem essa coluna correcta.

Na lista China-Angola importações a flag isAggregate parece não 
estar correcta até 2017 inclusive. 2018 já está bem.

See the [bug report](isaggregate_bug.ipynb)


In [19]:

df['valueForTotal'] = df['primaryValue']
df.loc[df['isAggregate'] == True,'valueForTotal'] = 0
df['subtotalAG2'] = df.groupby(['partnerCode','refYear','flowCode','cmdCodeAG2'])["valueForTotal"].transform('sum')
df['subtotalCmd'] = df.groupby(['partnerCode','refYear','flowCode','cmdCode'])["valueForTotal"].transform('sum')
df['percentAG2'] = df['primaryValue'] / df['subtotalAG2'] * 100
df['percentCmd'] = df['primaryValue'] / df['subtotalCmd'] * 100

In [20]:
import comtrade

excel_file_name = f"./reports/product_detail_{reporter_name}_{years_of_interest.replace(',','-')}_{flow}_{partner_name}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name, engine='xlsxwriter')

df2 = df[pco_cols_detail+['percentAG2']].sort_values('primaryValue',ascending=False)
df2.to_excel(excel_file, sheet_name='detail', index=True, startrow=2)

comtrade.excel_col_autowidth(df2, excel_file, 'detail')
comtrade.excel_format_currency(df2, excel_file,'detail', columns=['primaryValue'],width=20)
excel_file.close()
print(f"Excel file {excel_file_name} created")


Excel file ./reports/product_detail_China_2021_M_Brazil.xlsx created


In [21]:

pd.options.display.max_rows = 500
# df.sort_values(['partnerDesc','refYear','flowCode','subtotalAG2','cmdCodeAG2','primaryValue'],ascending=[True,True,True,False,True,False])[pco_cols_detail].head(500)
# format columns percentAG2 to percentage 3 decimals
df['percentAG2Form'] = df['percentAG2'].map("{:.3f}%".format)
df['percentCmdForm'] = df['percentCmd'].map("{:.3f}%".format)
df[df['isAggregate'] == False].sort_values(['reporterDesc','partnerDesc','refYear','cmdCode','subtotalAG2','percentAG2'],ascending=[True,True,True,True,False,False])\
    [['reporterDesc','partnerDesc','refYear','partner2Desc','percentCmdForm','valueForTotal','cmdCodeAG2','cmdCode','cmdDesc']+\
    ['customsDesc','motDesc','subtotalCmd','subtotalAG2']].head(500)


,reporterDesc,partnerDesc,refYear,partner2Desc,percentCmdForm,valueForTotal,cmdCodeAG2,cmdCode,cmdDesc,customsDesc,motDesc,subtotalCmd,subtotalAG2
2,China,Brazil,2021,World,100.000%,"4,620,523,121.00",02,020230,"Meat; of bovine animals, boneless cuts, frozen",All customs procedure codes,All modes of transport,"4,620,523,121.00","7,799,680,540.00"
4,China,Brazil,2021,World,100.000%,"39,241,333.00",02,020322,"Meat; of swine, hams, shoulders and cuts thereof, with bone in, frozen",All customs procedure codes,All modes of transport,"39,241,333.00","7,799,680,540.00"
5,China,Brazil,2021,World,100.000%,"1,617,075,329.00",02,020329,"Meat; of swine, n.e.c. in item no. 0203.2, frozen",All customs procedure codes,All modes of transport,"1,617,075,329.00","7,799,680,540.00"
7,China,Brazil,2021,World,100.000%,"2,784,082.00",02,020500,"Meat; of horses, asses, mules or hinnies, fresh, chilled or frozen",All customs procedure codes,All modes of transport,"2,784,082.00","7,799,680,540.00"
9,China,Brazil,2021,World,100.000%,"61,929,222.00",02,020649,"Offal, edible; of swine, (other than livers), frozen",All customs procedure codes,All modes of transport,"61,929,222.00","7,799,680,540.00"
11,China,Brazil,2021,World,100.000%,"229,346.00",02,020712,"Meat and edible offal; of fowls of the species Gallus domesticus, not cut in pieces, frozen",All customs procedure codes,All modes of transport,"229,346.00","7,799,680,540.00"
12,China,Brazil,2021,World,100.000%,"1,455,270,569.00",02,020714,"Meat and edible offal; of fowls of the species Gallus domesticus, cuts and offal, frozen",All customs procedure codes,All modes of transport,"1,455,270,569.00","7,799,680,540.00"
14,China,Brazil,2021,World,100.000%,"2,627,538.00",02,020910,"Fat; pig fat, free of lean meat, not rendered or otherwise extracted, fresh, chilled, frozen, sa...",All customs procedure codes,All modes of transport,"2,627,538.00","7,799,680,540.00"
87,China,Brazil,2021,World,100.000%,522.00,12,120110,"Soya beans; seed, whether or not broken",All customs procedure codes,All modes of transport,522.00,"33,120,876,740.00"
88,China,Brazil,2021,World,100.000%,"33,120,436,899.00",12,120190,"Soya beans; other than seed, whether or not broken",All customs procedure codes,All modes of transport,"33,120,436,899.00","33,120,876,740.00"
